In [ ]:
import os
import pandas as pd
import numpy as np
import random


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')


from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, ShuffleSplit, learning_curve, validation_curve
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler, LabelEncoder
import itertools
import timeit


In [ ]:
# from imblearn.over_sampling import RandomOverSampler

In [ ]:
# import dill
# # # from dill.settings import settings
# dill.dump_session('wilt-nt.db')
# # dill.load('done-seiwilt.db')

Data2

In [ ]:
# col_names = ['F1', 'F2', 'F3', 'class']

In [ ]:
df_wilt = pd.read_csv('wilt_tr.csv' )

print("Data has",len(df_wilt),"rows and", len(df_wilt.columns),"columns.")
if df_wilt.isnull().values.any():
    print("Data missing")
#df_bankn.head()
df_wilt.describe(include='all')

In [ ]:
df_wilt_test = pd.read_csv('wilt_test.csv' )

print("Data has",len(df_wilt),"rows and", len(df_wilt.columns),"columns.")
if df_wilt.isnull().values.any():
    print("Warning: Missing Data")
#df_bankn.head()
df_wilt.describe(include='all')

In [ ]:
sns.pairplot(df_wilt)

In [ ]:
plt.title("Wilt Data - Features Correlation")
plt.xlabel('Mean Red')
plt.ylabel('Mean Green')
plt.scatter(df_wilt['Mean_Red'], df_wilt['Mean_Green'] )

In [ ]:
df_wilt['Mean_Red'].corr(df_wilt['Mean_Green'])

In [ ]:
plt.title("Counts of Wilt Data Labels")
sns.countplot(df_wilt['class'])

In [ ]:
df_wilt['class'].value_counts()

In [ ]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(df_clean['y'])
# #le.transform(df_clean['y'])
# df_labels = df_wilt[['seismoacoustic','ghazard']].apply(LabelEncoder().fit_transform)
#df_labels.describe()


encoder = LabelEncoder().fit(df_wilt['class'])




In [ ]:

df_wilt['class'] = encoder.transform(df_wilt['class'])
df_wilt_test['class'] = encoder.transform(df_wilt_test['class'])


In [ ]:
# min_max_scaler = preprocessing.MinMaxScaler()
# names.insert(0, names.pop(names.index('y')))
# df_num = df_clean[names[1:]]
# x_scaled = min_max_scaler.fit_transform(df_num)
col_names = df_wilt.columns
print(col_names)

In [ ]:
scalar = MinMaxScaler()

In [ ]:
# df_scaled = pd.DataFrame(x_scaled, columns=names[1:])
# df_scaled.head()
# rem_names = ['genergy', 'gpuls', 'gdenergy',
#        'gdpuls', 'nbumps', 'nbumps2', 'nbumps3', 'nbumps4',
#        'nbumps5', 'nbumps6', 'nbumps7', 'nbumps89', 'energy', 'maxenergy',
#        'class']


scalar.fit(df_wilt[col_names[1:]])

x_scaled = scalar.transform(df_wilt[col_names[1:]])
x_scaled_test = scalar.transform(df_wilt_test[col_names[1:]])

# df_sc = pd.DataFrame(x_scaled, columns=col_names[1:-1])

In [ ]:
def get_tr_data():


    x = np.array(x_scaled)
    y = np.array(df_wilt.values[:,0]).astype('int')

    return x, y


def get_test_data():


    x = np.array(x_scaled_test)
    y = np.array(df_wilt_test.values[:,0]).astype('int')

    return x, y


In [ ]:
# Not splitting - this file is only for training
X_train, y_train = get_tr_data()
X_test, y_test = get_test_data()

In [ ]:
# df_clean.sort_index(axis=1, inplace=True)

In [ ]:
# df_clean.head()

In [ ]:
# sns.countplot(x='class',data=df_clean)

In [ ]:
# np.bincount(y_train), np.bincount(y_test)

In [ ]:
sns.countplot(df_wilt['class'])
# X_train, X_test, y_train, y_test = train_test_split(np.array(xBk),np.array(yBk), test_size=0.20)

In [ ]:
# ros = RandomOverSampler(random_state=1)

In [ ]:
# X_train , y_train = ros.fit_resample( X_train , y_train)

In [ ]:
sns.countplot(y_test)

### Resampling done only to training data. Testing data is still imbalanced

In [ ]:
# sns.countplot(y_test)

Now we have successfully loaded and processed both datasets. We are ready to start the ML!

### helper funcs

In [ ]:
#src: sklearn
def pllc(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 20)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("F1")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=-1, train_sizes=train_sizes, scoring='f1')
    


    
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    
    results ={'sizes':train_sizes,'tr_scores':train_scores_mean, 'val_scores': test_scores_mean, 'title':title}
    
    
    
    
    
    plt.grid(True)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="g")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="r")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="g",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="r",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.tight_layout()
    
    return plt, results

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

def tune_hp(estimator, X_train, y_train, title, param_name,param_range, xlabel,xvals=None, cv=5):

    
            
            
    train_scores, val_scores = validation_curve(estimator, X_train, y_train, param_name, 
                                                 param_range, cv=cv, n_jobs=-1, scoring='f1')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    #train_scores_std = np.std(train_scores, axis=1)
    val_scores_mean = np.mean(val_scores, axis=1)
    #test_scores_std = np.std(test_scores, axis=1)
    
    if xvals is not None:
        param_range=xvals
            
    plt.grid(True)  
    plt.plot(param_range, train_scores_mean, 'o-', color = 'g', label='Train Score')
    plt.plot(param_range, val_scores_mean, 'o-', color='r', label='Validation Score')
    plt.ylabel('F1')
    plt.xlabel(xlabel)
    
    plt.title(title)
    plt.legend(loc='best')
#     plt.tight_layout()


# ML- DT

In [ ]:
tune_hp(estimator=DecisionTreeClassifier(random_state=1), X_train= X_train, y_train=y_train, title="Hypertunning Decision Min Tree Leaf Size - Wilt Data",
        param_name='min_samples_leaf',param_range=np.arange(1,30,1), xlabel="Min Leaf Size", cv=5)

In [ ]:
param_grid_DT = {'min_samples_leaf':np.array([1,2,3]), 'max_depth':np.arange(3,8)}

In [ ]:
GSTree = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=param_grid_DT, cv=10, scoring='f1', n_jobs=-1)
GSTree.fit(X_train, y_train)
max_depth, min_samples_leaf =GSTree.best_params_['max_depth'], GSTree.best_params_['min_samples_leaf']
print("Tree chosen parameters: ")
print(GSTree.best_params_)

# Tree Grid Search chosen parameters: 
# {'min_samples_leaf': 1, 'max_depth': 28}

In [ ]:
title = "Learning Curve DT- Wilt Data"

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)

estimator =  DecisionTreeClassifier(criterion='gini',max_depth=7, min_samples_leaf= 1, random_state=1)
_ , DTLC_results = pllc(estimator, title, X_train, y_train, ylim=None, cv=cv, n_jobs=-1)

In [ ]:

DTLC_results

# NN

In [ ]:
mlp = MLPClassifier(solver='adam', random_state=1, verbose=10, activation='relu'
                    , learning_rate_init=0.01 ,)


tune_hp(estimator=mlp, X_train= X_train, y_train=y_train, title="NN Hypertunning No. nodes in hidden layer -Wilt Data",
        param_name='hidden_layer_sizes',param_range=[(10,),(40,),(70,),(75,),(80,),(85,),(90,),(100,),(150,) ], 
        xlabel="No. nodes in hidden layer", xvals=None , cv=10)


In [ ]:
# param_grid_nn = {'learning_rate_init':[0.01,.001, .1]}
mlp1 = MLPClassifier(solver='adam',random_state=1, verbose=0,learning_rate_init=.01 , hidden_layer_sizes=(80,), activation='relu')

In [ ]:
# GSmlp = GridSearchCV(estimator = mlp1, param_grid=param_grid_nn, cv=5, scoring='f1', n_jobs=-1)
# GSmlp.fit(X_train, y_train)
# learning_rate_init =GSmlp.best_params_['learning_rate_init']
# print("Tree Grid Search chosen parameters: ")
# print(GSmlp.best_params_)
mlp1.fit(X_train, y_train)
print(mlp1.loss_curve_)

In [ ]:
title = "Learning Curve NN- Wilt Data"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)

# estimator =  MLPClassifier(solver='adam',random_state=1, verbose=10, hidden_layer_sizes=(50,300,),
#                           learning_rate_init= 0.01, activation= 'relu')
_ , NNLC_results = pllc(mlp1, title, X_train, y_train, ylim=None, cv=cv, n_jobs=-1)

In [ ]:
NNLC_results

The final section for neural network will plot the loss curve for each dataset over the iterations.

In [ ]:
iters = np.arange(1,len(mlp1.loss_curve_)+1).astype(int)
print(iters)

In [ ]:


plt.figure()
plt.title("NN- Loss Curve - Wilt Data")
plt.xlabel("Number of Iterations")
plt.ylabel("Log Loss")
plt.plot(iters, mlp1.loss_curve_, 'o-', markersize=4)


#### Boosted

In [ ]:
GBC = GradientBoostingClassifier( random_state=1)

#tuneBoosted(X_train, y_train, X_test, y_test, 10, 1, title="N of Estimators: Boosted Tree complexity curve")

In [ ]:

tune_hp(estimator=GBC, X_train= X_train, y_train=y_train, title="Hypertunning GradBoosted Tree depth (estimators=100) - Wilt Data",
        param_name='max_depth',param_range=np.linspace(1,10,10).astype('int'), xlabel="Max Depth")

In [ ]:
param_grid = {'min_samples_leaf': np.array([1]),
              'max_depth': [4],
              'n_estimators': np.linspace(50,100,10).round().astype('int')}

boost = GridSearchCV(estimator = GBC, param_grid=param_grid, cv=10, n_jobs=-1)
boost.fit(X_train, y_train)
print(" best parameters are:")
print(boost.best_params_)

In [ ]:
estimator_GB =  GradientBoostingClassifier(max_depth=4 , min_samples_leaf=1 , n_estimators=78,random_state=1,)
estimator_GB.fit(X_train, y_train)

In [ ]:
title = 'Learning Curve for Grad-Boosted- Wilt Data'
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)


_ , GBDTLC_results = pllc(estimator_GB, title, X_train, y_train, ylim=None, cv=cv, n_jobs=-1)

In [ ]:
GBDTLC_results

# SVM

In [ ]:
scaler_st = StandardScaler()
scaler_st = scaler_st.fit(X_train)

In [ ]:
#scaler = StandardScaler()
#GBC = GradientBoostingClassifier( random_state=1,)




    
X_train_sc = scaler_st.transform(X_train)
X_test_sc = scaler_st.transform(X_test)


In [ ]:
tune_hp(estimator=SVC(random_state=1,kernel='rbf', C=15), X_train= X_train_sc, y_train=y_train, title="Hypertunning SVM Gamma- Wilt Data",
        param_name='gamma',param_range=[0.05, .1,.2,.3,.4,1], xlabel="Gamma", cv=10)

In [ ]:
title = "Learning Curve SVM- Wilt Data"


cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)

estimator =  SVC(random_state=1, kernel='rbf', C=15, gamma=0.2)
_ , SVM_results = pllc(estimator, title, X_train_sc, y_train, ylim=None, cv=cv, n_jobs=-1)

In [ ]:
SVM_results

# KNN

In [ ]:

tune_hp(estimator=KNeighborsClassifier(n_jobs=-1), X_train= X_train, y_train=y_train, title="Hypertunning KNN # of neighbors- Wilt Data",
        param_name='n_neighbors',param_range=[1,2,3,4,5,6], xlabel="K-neighbors", cv=10)

In [ ]:
title = "Learning Curve KNN- Wilt Data"


cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1)

estimator =  KNeighborsClassifier(n_jobs=-1, n_neighbors=1,)
_ , KNN_results = pllc(estimator, title, X_train, y_train, ylim=None, cv=cv, n_jobs=-1)

In [ ]:
KNN_results

# comparison plots

In [ ]:

#results = [DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results]

def plot_LRs(sizes,DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, title,scr):
    
    plt.figure()
    plt.title(title)
    plt.xlabel("Training Examples")
    plt.ylabel("Model F1 Score")
    plt.plot(sizes, DTLC_results[scr], '-', color="r", label="DT")
    plt.plot(sizes, NNLC_results[scr] , '-', color="g", label="NN")
    plt.plot(sizes, GBDTLC_results[scr] , '-', color="b", label="Grad-Bosted")
    plt.plot(sizes, SVM_results[scr] , '-', color="k", label="SVM")
    plt.plot(sizes, KNN_results[scr] , '-', color="y", label="KNN")
    plt.legend(loc="best")
    #plt.show() 

In [ ]:
plot_LRs(DTLC_results['sizes'],DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, "Validation Learning Rates - Wilt Data",'val_scores')

In [ ]:
plot_LRs(DTLC_results['sizes'],DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, "Validation Learning Rates - Wilt Data",'val_scores')

In [ ]:
plot_LRs(DTLC_results['sizes'],DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, "Training Learning Rates - Wilt Data",'tr_scores')

In [ ]:
plot_LRs(DTLC_results['sizes'],DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, "Training Learning Rates - Wilt Data",'tr_scores')

In [ ]:
plot_LRs(DTLC_results['sizes'],DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, "Validation Learning Rates - Wilt Data",'val_scores')

In [ ]:

results = [DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results]

print(results)
# def plot_LRs(n,DTLC_results, NNLC_results, GBDTLC_results, SVM_results, KNN_results, title):
    
#     plt.figure()
#     plt.title("Model Learning Rates: " + title)
#     plt.xlabel("Training Examples")
#     plt.ylabel("F1_macro")
#     plt.plot(n, NNlearn, '-', color="b", label="Neural Network")
#     plt.plot(n, SMVlearn, '-', color="r", label="SVM")
#     plt.plot(n, kNNlearn, '-', color="g", label="kNN")
#     plt.plot(n, DTlearn, '-', color="m", label="Decision Tree")
#     plt.plot(n, BTlearn, '-', color="k", label="Boosted Tree")
#     plt.legend(loc="best")
#     plt.show() 

## Testing

In [ ]:
def testing_func(clf,X_train, X_test, y_train, y_test):
    
    start_time = timeit.default_timer()
    clf.fit(X_train, y_train)
    end_time = timeit.default_timer()
    training_time = end_time - start_time
    
    start_time = timeit.default_timer()    
    y_pred = clf.predict(X_test)
    end_time = timeit.default_timer()
    pred_time = end_time - start_time
    

    f1 = f1_score(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    cm = confusion_matrix(y_test,y_pred )
    
    df_cm = pd.DataFrame(cm, index= ['n', 'w'] , columns= ['n' , 'w'])


    
    print("Model Training Time (s):   "+"{:.5f}".format(training_time))
    print("Model Prediction Time (s): "+"{:.5f}\n".format(pred_time))
    print("F1 Score:  "+"{:.2f}".format(f1))
    print("Accuracy:  "+"{:.2f}".format(accuracy))

    print("Precision: "+"{:.2f}".format(precision))
    print("Recall:  "+"{:.2f}".format(recall))

    
    sns.heatmap(df_cm,cmap="YlGnBu", annot=True, fmt="d")

In [ ]:
f_DT  = DecisionTreeClassifier(criterion='gini',max_depth=5, min_samples_leaf= 1, random_state=1)
f_nn = MLPClassifier(solver='adam',random_state=1, hidden_layer_sizes=(80,),
                          learning_rate_init= 0.01, activation= 'relu')
f_GBC = GradientBoostingClassifier(max_depth=4 , min_samples_leaf=1 , n_estimators=78,random_state=1,)
f_svm = SVC(random_state=1, kernel='rbf', C=15, gamma=0.2)
f_KNN = KNeighborsClassifier(n_jobs=-1, n_neighbors=1,)


In [ ]:
os.getcwd()

In [ ]:
# import pipreqs

In [ ]:
# pipreqs -print

In [ ]:
testing_func(f_DT,X_train, X_test, y_train
                            , y_test)

In [ ]:
testing_func(f_nn,X_train, X_test, y_train, y_test)

In [ ]:

testing_func(f_GBC,X_train, X_test, y_train, y_test)



In [ ]:
testing_func(f_svm,X_train_sc, X_test_sc, y_train, y_test)

In [ ]:
testing_func(f_KNN,X_train, X_test, y_train, y_test)

In [ ]:
testing_func(DecisionTreeClassifier(max_depth=6, min_samples_leaf=3),X_train, X_test, y_train, y_test)